# CV 7

 to install netcat:
    apt update
    apt-get install netcat
    nc -lkp 9999 &

In [5]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col,regexp_replace

spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()



In [4]:
spark.stop()

In [6]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col,regexp_replace,current_timestamp, window, desc

spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()
lines = spark.readStream.option('newFilesOnly', 'true').text('./data')
#lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

words = lines.select(explode(split(lines.value, " ")).alias("word"))

words = words.select(lower(col('word')).alias('word'))
words = words.withColumn('word', regexp_replace('word', r'[^\s\w]', ''))
words = words.filter("word != ''")

words = words.withColumn('eventTime', current_timestamp())
wordCounts = words.groupBy(window(col("eventTime"), "30 seconds", "15 seconds").alias('start'), col("word")).count()
wordSorted = wordCounts.orderBy("start",desc("count"))


query = wordSorted.writeStream.outputMode("complete").format("memory").queryName("my_query").start()

while True:
    #spark.sql_scripts("SELECT * FROM my_query").show(truncate=False)
    # just show 1 window each time
    latest_window_query = """
    SELECT *
    FROM my_query
    WHERE start = (SELECT MAX(start) FROM my_query)
    """

    spark.sql(latest_window_query).show(truncate=False)

    time.sleep(5)


23/12/15 10:30:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5b495670-93a2-4cd1-816f-36a6b6cdea3e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/15 10:30:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----+----+-----+
|start|word|count|
+-----+----+-----+
+-----+----+-----+


+-----+----+-----+
|start|word|count|
+-----+----+-----+
+-----+----+-----+
+------------------------------------------+--------+-----+
|start                                     |word    |count|
+------------------------------------------+--------+-----+
|{2023-12-15 10:30:15, 2023-12-15 10:30:45}|text    |2    |
|{2023-12-15 10:30:15, 2023-12-15 10:30:45}|stffysaf|1    |
+------------------------------------------+--------+-----+


KeyboardInterrupt: 

# cv 7 bonus

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, current_timestamp, lower, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

# Inicializace SparkSession
spark = SparkSession.builder.appName("WordCountTxtFileStreaming").getOrCreate()

# Definice schématu pro stream souborů
schema = StructType([StructField("value", StringType()), StructField("timestamp", TimestampType())])

# Definice schématu pro stream souborů
lines = spark.readStream.format("csv") \
    .schema(schema) \
    .option("header", "false") \
    .option("latestFirst", "true") \
    .option("maxFilesPerTrigger", 1) \
    .option("filter", "input_file_name LIKE '%.txt'") \
    .load("./data/*")

# Přidání sloupce timestamp
lines_with_timestamp = lines.withColumn("timestamp", current_timestamp())

# Převedení na malá písmena a odstranění interpunkce
cleaned_lines = lines_with_timestamp.withColumn("cleaned_value", regexp_replace(lower(lines_with_timestamp.value), "[^a-z\\s]", ""))

# Rozdělení řádků na slova
words = cleaned_lines.select(explode(split(cleaned_lines.cleaned_value, " ")).alias("word"), "timestamp")

# Definice schématu pro stream
word_counts = words.groupBy("word").count()

# Výpis výsledků na konzoli
query = word_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Čekání na ukončení streaming query
query.awaitTermination()

23/12/15 10:35:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3b337c8c-2b90-4aa6-9c7d-4fcc7aaaedd5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/15 10:35:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/15 10:35:29 WARN FileStreamSource: 'latestFirst' is true. New files will be processed first, which may affect the watermark
value. In addition, 'maxFileAge' will be ignored.


-------------------------------------------
Batch: 3
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|   hokus|    1|
|  txtetx|    1|
|stffysaf|    1|
|    text|    6|
|   pokus|    1|
+--------+-----+


-------------------------------------------
Batch: 4
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|   hokus|    1|
|stffysaf|    1|
|    text|    4|
|   pokus|    1|
+--------+-----+


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hokus|    1|
| text|    2|
|pokus|    1|
+-----+-----+


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hokus|    1|
|  txt|    1|
| text|    2|
|pokus|    1|
+-----+-----+


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/rob/Programming/DataScience/venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rob/Programming/DataScience/venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 